In [1]:
import pickle
from neo4j import GraphDatabase
from dotenv import dotenv_values
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cdlib import algorithms, viz, evaluation

from hp_nlp_graph.neo4j import add_metrics_to_neo4j

Note: to be able to use all crisp methods, you need to install some additional packages:  {'bayanpy', 'wurlitzer', 'graph_tool', 'infomap'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'ASLPAw', 'pyclustering'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap'}


In [2]:
book_number = 4
interactions_by_chapter = pickle.load(
    open(f"data/processed/{book_number}/interactions_by_chapter.pkl", "rb")
)

In [3]:
neo4j_config = dotenv_values(".env")
driver = GraphDatabase.driver(
    neo4j_config["NEO4J_URL"],
    auth=(neo4j_config["NEO4J_USER"], neo4j_config["NEO4J_PASSWORD"]),
)

In [4]:
dfs = []
for chapter, distances in interactions_by_chapter.items():
    df = pd.DataFrame.from_dict(
        dict(distances), orient="index", columns=["weight"]
    ).reset_index(names=["characters"])
    df[["source", "target"]] = pd.DataFrame(df.characters.tolist(), index=df.index)
    df["chapter"] = chapter
    df["book"] = 1
    df = df[["source", "target", "weight", "chapter", "book"]]
    dfs.append(df)
df = (
    pd.concat(dfs)
    .groupby(["source", "target"])
    .weight.sum()
    .sort_values(ascending=False)
    .reset_index()
)

In [5]:
G = nx.from_pandas_edgelist(df, "source", "target", ["weight"], create_using=nx.Graph())

In [6]:
eigen_centrality = nx.eigenvector_centrality(G, weight="weight")
betweenness_centrality = nx.betweenness_centrality(G, weight="weight")
degree_centrality = nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
pagerank = nx.pagerank(G, weight="weight")
hub_centrality, authority_centrality = nx.hits(G)
degree = dict(nx.degree(G))
weighted_degree = dict(nx.degree(G, weight="weight"))

metrics = {
    "eigen_centrality": eigen_centrality,
    "betweenness_centrality": betweenness_centrality,
    "degree_centrality": degree_centrality,
    "closeness_centrality": closeness_centrality,
    "pagerank": pagerank,
    "hub": hub_centrality,
    "authority": authority_centrality,
    "degree": degree,
    "weighted_degree": weighted_degree,
}
metrics_df = pd.DataFrame.from_dict(metrics)
metrics_df.index.name = "name"
metrics_df

,eigen_centrality,betweenness_centrality,degree_centrality,closeness_centrality,pagerank,hub,authority,degree,weighted_degree
name,,,,,,,,,
Harry Potter,0.612403,0.140173,0.700565,0.751513,0.195312,1.717248e-01,1.717248e-01,124,10019
Ronald Weasley,0.542606,0.097434,0.514124,0.652859,0.094660,1.521447e-01,1.521447e-01,91,5173
Hermione Granger,0.457907,0.027906,0.440678,0.610435,0.070491,1.283980e-01,1.283980e-01,78,3959
Albus Dumbledore,0.155308,0.066260,0.344633,0.581067,0.036063,4.354778e-02,4.354778e-02,61,1830
Cedric Diggory,0.100089,0.016410,0.254237,0.545453,0.017857,2.806432e-02,2.806432e-02,45,890
...,...,...,...,...,...,...,...,...,...
Basil,0.000022,0.000000,0.005650,0.345082,0.000860,6.174334e-06,6.174334e-06,1,1
Fawcett,0.000021,0.000000,0.011299,0.351539,0.000917,5.940010e-06,5.940010e-06,2,2
Otto Bagman,0.000015,0.000000,0.005650,0.353746,0.000862,4.106873e-06,4.106873e-06,1,1


In [7]:
louvain_comms = algorithms.louvain(G, weight="weight")
louvain = pd.DataFrame.from_dict(
    dict(louvain_comms.to_node_community_map()),
    orient="index",
    columns=["louvain"],
)
louvain.index.name = "name"

In [8]:
leiden_comms = algorithms.leiden(G, weights="weight")
leiden = pd.DataFrame.from_dict(
    dict(leiden_comms.to_node_community_map()),
    orient="index",
    columns=["leiden"],
)
leiden.index.name = "name"

In [9]:
girvan_newman_comms = algorithms.girvan_newman(G, level=5)
girvan_newman = pd.DataFrame.from_dict(
    dict(girvan_newman_comms.to_node_community_map()),
    orient="index",
    columns=["girvan_newman"],
)
girvan_newman.index.name = "name"

In [10]:
spectral_comms = algorithms.spectral(G, kmax=8)
spectral = pd.DataFrame.from_dict(
    dict(spectral_comms.to_node_community_map()),
    orient="index",
    columns=["spectral"],
)
spectral.index.name = "name"

In [11]:
metrics_df = metrics_df.join([louvain, leiden, girvan_newman, spectral])

In [12]:
metrics_df.to_csv(f"data/processed/{book_number}/metrics.csv")

In [13]:
add_metrics_to_neo4j(driver, metrics_df.reset_index().to_dict("records"))

In [13]:
for col in metrics_df.columns:
    nx.set_node_attributes(G, metrics_df[col].to_dict(), col)

In [14]:
nx.write_gexf(G, f"data/processed/{book_number}/graph.gexf")